In [ ]:
from ipywidgets import Button, IntSlider, interactive_output, HBox, Output, Box
from itertools import product, permutations
from collections import defaultdict

import numpy as np
import pandas as pd
from amplify import (
    Solver,
    decode_solution,
    gen_symbols,
    BinaryPoly,
    sum_poly,
    BinaryQuadraticModel,
)
from amplify.client import FixstarsClient
from amplify.constraint import equal_to, penalty, greater_equal

In [ ]:
%%html
<style>.wariate {
    width: 100%;
    margin: 0 auto;
    font-size: 100%;
    font-size: 1.vw;
}


# Employee Assignment Problem

Optimal assignment of employees to stores is an important task for retail and service industries with a large number of employees. Typically, such assignment processes must take into account each employee's position, skills, and preferred work location, as well as a variety of daily-changing tasks in each store.

Here, we address the combinatorial optimization problem of employee assignment, using a restaurant chain as an example. In particular, we aim to assign employees appropriately according to position, skill type and level, position, and desired work location.

For example, an employee may have the following attributes:

**Position**
- Store manager
- Assistant manager
- Staff (no position)

**Role**
- Kitchen staff
- Floor staff

**Skill level**
- Cooking Skills  
  For a sushi chain restaurant considered in [Step 3](#step3):
  - Filleting skill
  - *Nigiri* (hand role) skill
  - Soup cooking skill
  - A la carte dish preparation skill

**Work location preference**
- For each work location, an employee has one of the following preference:
  - Unavailable
  - Available
  - Preferred

In addition, suppose that the following requirements must be met at each store.

- **Requirements**
  - Number of people required for each position and role
  - Type and level of culinary skills required

Here, we attempt to assign all employees to stores based on the above requirements and attributes. In addition, employee assignments are made to minimize the variation in fill rate among the stores.

Since it is complex to consider all the requirements at once, we implement the solver following the steps.

In [ ]:
class BaseDemo(object):
    width = 60
    problem_out = None
    problem_result_out = None

    def __init__(self):
        super().__init__()
        run_btn = Button(
            description="Run", button_style="", tooltip="Run", icon="check"
        )
        run_btn.on_click(self.show_result)
        self.run_btn = run_btn

    def show_problem(self, *args, **kwargs):
        assert self.problem_out is not None
        with self.problem_out:
            print("INPUT".center(self.width, "="))
            self._show_problem(*args, **kwargs)

    def show_result(self, *args, **kwargs):
        assert self.problem_result_out is not None
        with self.problem_result_out:
            self.problem_result_out.clear_output()
            print("OUTPUT".center(self.width, "="))
            self._show_result(*args, **kwargs)

    def _show_problem(self, *args, **kwargs):
        """入力結果を可視化する関数"""
        raise NotImplementedError

    def _show_result(self, *args, **kwargs):
        """実行結果を可視化する関数"""
        raise NotImplementedError

    def __str__(self):
        s = "Amplifyを用いた"
        return s

In [ ]:
class AmplifyProblem(object):
    def construct(self):
        raise NotImplementedError

    def solve(self):
        raise NotImplementedError

    def _solve(self, model):
        # クライアントの設定
        client = FixstarsClient()
        # client.token = "xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx"  # ローカル環境では Amplify AEのアクセストークンを入力してください
        client.parameters.timeout = 1000  #  タイムアウト１秒
        # ソルバーを定義して実行
        solver = Solver(client)
        result = solver.solve(model)
        return result

---
## **Step 1**

In this step, employees are assigned to each store, based on each employee's work location preference and the number of employees required for each store.

Suppose each store requires several employees, while each employee has a work preference for each store. Their preference levels are expressed as integers between 0 and 2 for each store, corresponding to the following: unavailable, available, and preferred.

- Unavailable → Preference level: 0
- Available → Preference level: 1
- Preferred → Preference level: 2

Our goal is to assign employees to stores in a way that matches the number of employees to the needs of the stores, while satisfying employees' location preferences as much as possible.

Clicking the "Run" button outputs the result of the assignment for each employee and for each store.

In [ ]:
class DemoStep1(BaseDemo, AmplifyProblem):
    name = "step1"

    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        problem_out = Output()
        problem_result_out = Output()
        problem_out.add_class(self.name)
        problem_result_out.add_class(self.name)
        self.problem_out = problem_out
        self.problem_result_out = problem_result_out

    def _show_problem(self, *args, **kwargs):
        ## store_require.csv
        # 各店舗の要求人数情報の読み込み
        dict_req = dict(location=["tenjin", "hakata"], employee=[2, 3])
        df_req = pd.DataFrame.from_dict(dict_req, orient="index").T
        message = ("The number of people requested for each store is shown below.",)
        print("\n".join(message))
        display(df_req)

        # 各従業員の勤務希望情報
        dict_worker_loc = dict(
            worker_id=[0, 1, 2, 3, 4], tenjin=[2, 2, 1, 0, 1], hakata=[1, 1, 1, 1, 0]
        )

        df_worker_loc = pd.DataFrame.from_dict(dict_worker_loc, orient="index").T
        message = ("Each employee's work location preference is shown below.",)
        print("\n".join(message))
        display(df_worker_loc)

        self.df_req = df_req
        self.df_worker_loc = df_worker_loc

    def _show_result(self, *args, **kwargs):
        print("The results output where each employee will work.")
        df_result = self.solve(self, *args, **kwargs)
        display(df_result)

        num_member = df_result["location"].value_counts()
        fill_rate = self.df_req.copy()
        fill_rate["fill_rate"] = [
            self.df_req.loc[l]["employee"] / num_member[self.idx2loc[l]]
            for l in range(self.num_locations)
        ]
        print("Verify that the number of employees required by the store is being met.")
        display(fill_rate)

    def solve(self, *args, **kwargs):
        """問題を解く関数"""
        model, location_variables = self.construct(
            self.df_worker_loc,
            self.df_req,
        )
        result = self._solve(model)
        df_result = self.decode(
            result, location_variables, self.df_worker_loc, self.df_req, self.locations
        )
        return df_result

    def construct(self, df_worker_loc, df_req, *args, **kwargs):
        # =======
        # 対応関係
        # =======
        # dictの作成
        idx2loc = dict((i, v) for i, v in enumerate(df_req["location"].values))
        loc2idx = dict((v, i) for i, v in enumerate(df_req["location"].values))

        self.idx2loc = idx2loc
        self.loc2idx = loc2idx

        ## 店舗名の取得
        workers = df_worker_loc["worker_id"].values
        locations = df_req["location"].values

        self.workers = workers
        self.locations = locations

        ## 各データ長を取得
        num_workers = len(workers)
        num_locations = len(locations)
        self.num_workers = num_workers
        self.num_locations = num_locations

        # =======
        # 変数定義
        # =======
        # 従業員iが役職jで店舗lに勤務することを表す変数
        location_variables = gen_symbols(BinaryPoly, num_workers, num_locations)

        # =======
        # 変数固定
        # =======
        ## 勤務不可能地域に関しては変数を定数化
        from itertools import product

        for i, l in product(range(num_workers), locations):
            worker_req = df_worker_loc.iloc[i][l]
            if worker_req == 0:
                # 勤務不可
                location_variables[i][loc2idx[l]] = BinaryPoly(0)

        ## 充足率の計算
        from amplify import sum_poly

        w = [
            (sum_poly(num_workers, lambda i: location_variables[i][l]))
            / df_req["employee"][l]
            for l in range(num_locations)
        ]

        # =========
        # 目的関数定義
        # =========
        # 充足率の平均の最大化
        average_fill_rate_cost = -((sum_poly(w) / len(w)) ** 2)

        # 充足率の分散の最小化
        variance_fill_rate_cost = (
            sum_poly(len(w), lambda i: w[i] ** 2) / len(w)
            - (sum_poly(w) / num_locations) ** 2
        )

        # 従業員の希望度最大化
        location_cost = -sum_poly(
            num_workers,
            lambda i: sum_poly(
                num_locations,
                lambda l: df_worker_loc.loc[i][idx2loc[l]] * location_variables[i][l],
            ),
        )

        # ==========
        # 制約条件定義
        # ==========
        # 従業員iは同時に1店舗のみ勤務できる
        location_constarints = sum(
            [equal_to(sum_poly(location_variables[i]), 1) for i in range(num_workers)]
        )

        # 店舗の合計人数は要求人数以上
        require_constraints = sum(
            [
                greater_equal(
                    sum_poly(num_workers, lambda i: location_variables[i][l]),
                    df_req["employee"][l],
                )
                for l in range(num_locations)
            ]
        )

        # ============
        # 最適化モデル作成
        # ============
        # それぞれの目的関数の係数
        loc_priority = 1
        ave_fill_priority = 1
        var_fill_priority = 10

        # 目的関数
        cost_func = (
            loc_priority * location_cost
            + ave_fill_priority * average_fill_rate_cost
            + var_fill_priority * variance_fill_rate_cost
        )

        # 制約条件を表すペナルティ関数の重み
        constraint_weight = 10

        # 制約条件
        constraints = constraint_weight * (location_constarints + require_constraints)

        # 最適化モデル
        model = cost_func + constraints
        return model, location_variables

    def decode(
        self,
        result,
        location_variables,
        df_worker_loc,
        df_req,
        locations,
        *args,
        **kwargs,
    ):
        # 制約条件チェック
        if len(result) == 0:
            raise RuntimeError("The given constraints are not satisfied")
        values = result[0].values
        energy = result[0].energy

        # 勤務地に関する変数の解
        location_solutions = decode_solution(location_variables, values, 0)

        location_index_list = np.where(np.array(location_solutions) == 1)[1]
        dict_df = defaultdict(list)

        for i, loc_ind in enumerate(location_index_list):
            ## 配属勤務地
            worker_id = df_worker_loc.loc[i]["worker_id"]
            loc = locations[loc_ind]
            dict_df["worker_id"].append(worker_id)
            dict_df["location"].append(loc)

        df_result = pd.DataFrame.from_dict(dict_df, orient="index").T
        return df_result

In [ ]:
# インスタンス化
problem = DemoStep1()
display(HBox([problem.run_btn]), HBox([problem.problem_out]))
problem.show_problem()

display(HBox([problem.problem_result_out]))

<br>
<br>

---

## **Step 2**

In Step 2, in addition to "*each employee's preference for work location*" and "*the number of employees required for each store*" considered in **Step 1**, we will assign employees to satisfy the number of employees for each **position** required at each store.

For example, there may be a store that requires one manager and three staff members.

We consider three positions which are *store manager*, *assistant manager*, and *staff*. Each employee has an attribute that indicates whether they can be assigned to each position, in addition to the work location preference mentioned above.

As inputs, the required numbers of employees, managers and submanagers for each store are stored below. Clicking the "Run" button outputs the result of the assignment for each employee and for each store.

In [ ]:
class DemoStep2(BaseDemo, AmplifyProblem):
    name = "step2"

    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        problem_out = Output()
        problem_result_out = Output()
        problem_out.add_class(self.name)
        problem_result_out.add_class(self.name)
        self.problem_out = problem_out
        self.problem_result_out = problem_result_out

    def _show_problem(self, *args, **kwargs):
        # 各店舗の要求スキル情報の読み込み
        ## store_require.csv
        # 各店舗の要求役割情報
        dict_req = dict(
            location=["tenjin", "hakata", "akasaka", "gakken"],
            manager=[1, 1, 1, 1],
            submanager=[1, 0, 1, 1],
            employee=[2, 2, 2, 2],
        )
        df_req = pd.DataFrame.from_dict(dict_req, orient="index").T
        message = (
            "The number of requests for manager and submanager for each store is shown below.",
        )
        print("\n".join(message))
        display(df_req)

        # 各従業員の勤務希望情報
        dict_worker_loc = dict(
            worker_id=[0, 1, 2, 3, 4, 5, 6, 7, 8],
            tenjin=[2, 0, 0, 0, 1, 1, 2, 1, 1],
            hakata=[1, 0, 0, 2, 2, 2, 1, 2, 1],
            akasaka=[1, 0, 0, 1, 0, 1, 1, 1, 2],
            gakken=[1, 2, 2, 0, 0, 0, 0, 0, 0],
        )
        df_worker_loc = pd.DataFrame.from_dict(dict_worker_loc, orient="index").T
        message = ("Each employee's work location preference is shown below.",)
        print("\n".join(message))
        display(df_worker_loc)

        # 各従業員のスキル情報の読み込み
        dict_worker_skill = dict(
            worker_id=[0, 1, 2, 3, 4, 5, 6, 7, 8],
            manager=[1, 1, 0, 0, 1, 1, 1, 0, 1],
            submanager=[1, 1, 1, 0, 1, 1, 1, 0, 1],
            employee=[1, 1, 1, 1, 1, 1, 1, 1, 1],
        )
        df_worker_skill = pd.DataFrame.from_dict(dict_worker_skill, orient="index").T

        message = (
            "Below shows the skill information of each employee.",
            "For managers and submanagers, if the value is 1, it ",
            "means that the employee is available for the position. ",
        )
        print("\n".join(message))
        display(df_worker_skill)

        self.df_req = df_req
        self.df_worker_loc = df_worker_loc
        self.df_worker_skill = df_worker_skill

    def _show_result(self, *args, **kwargs):
        print(
            "The output of the results, where each employee will work from the results."
        )
        df_result = self.solve(self, *args, **kwargs)
        display(df_result)

        message = (
            "To check how well the required number of employees is met for each store, ",
            "we output the fillrate. Here, where a cell in the table has a component of ",
            "`None`, it means that the required number of employees for that role is zero.",
        )
        print("\n".join(message))
        dict_result_loc = defaultdict(lambda: defaultdict(int))
        for loc, role in product(self.locations, self.roles):
            dict_result_loc[loc][role] = 0

        for i in range(len(df_result)):
            data = df_result.loc[i]
            role = data["role"]
            location = data["location"]
            dict_result_loc[location][role] += 1

            if role != "employee":
                dict_result_loc[location]["employee"] += 1

        df_result_loc = pd.DataFrame.from_dict(dict_result_loc, orient="index")
        dict_result_loc = defaultdict(defaultdict)
        for i in range(len(df_result_loc)):
            data = df_result_loc.iloc[i]
            loc = data.name
            for role in self.roles:
                num_req = self.df_req[self.df_req["location"] == loc][role].item()
                num_wariate = data[role].item()
                dict_result_loc[loc][
                    f"{role}_fillrate"
                ] = f"{num_wariate/num_req if num_req > 0 else None}"

        df_result_loc = pd.DataFrame.from_dict(dict_result_loc, orient="index")
        display(df_result_loc.style)

    def solve(self, *args, **kwargs):
        """問題を解く関数"""
        model, location_variables, role_variables = self.construct(
            self.df_worker_loc, self.df_req, self.df_worker_skill
        )
        result = self._solve(model)
        df_result = self.decode(
            result,
            location_variables,
            role_variables,
            self.df_worker_loc,
            self.df_worker_skill,
            self.df_req,
            self.roles,
            self.locations,
        )
        return df_result

    def construct(self, df_worker_loc, df_req, df_worker_skill, *args, **kwargs):
        # =======
        # 対応関係
        # =======
        # 従業員id
        workers = df_worker_loc["worker_id"].values
        # 店舗名
        locations = df_req["location"].values
        # 役職名
        roles = ["manager", "submanager", "employee"]

        self.roles = roles
        self.locations = locations
        self.workers = workers

        # dictの作成
        idx2loc = dict((i, v) for i, v in enumerate(locations))
        loc2idx = dict((v, i) for i, v in enumerate(locations))
        idx2role = dict((i, v) for i, v in enumerate(roles))
        role2idx = dict((v, i) for i, v in enumerate(roles))

        # 各データ長を取得
        num_workers = len(workers)
        num_locations = len(locations)
        num_roles = len(roles)

        # =======
        # 変数定義
        # =======
        # 従業員iが役職jで店舗lに勤務することを表す変数
        role_variables = gen_symbols(BinaryPoly, num_workers, num_roles, num_locations)

        # =======
        # 変数固定
        # =======
        for i, l in product(range(num_workers), locations):
            worker_req = df_worker_loc.iloc[i][l]
            if worker_req == 0:
                # 全ての役職で店舗割当が不可
                for j in range(num_roles):
                    role_variables[i][j][loc2idx[l]] = BinaryPoly(0)

        for i, j in product(range(num_workers), roles):
            worker_skill = df_worker_skill.iloc[i][j]
            if worker_skill == 0:
                # 全ての店舗で役職が不可
                for l in range(num_locations):
                    role_variables[i][role2idx[j]][l] = BinaryPoly(0)

        location_variables = [
            [
                sum_poly(num_roles, lambda j: role_variables[i][j][l])
                for l in range(num_locations)
            ]
            for i in range(num_workers)
        ]

        ## 充足率の計算
        w = [
            (sum_poly(num_workers, lambda i: location_variables[i][l]))
            / df_req["employee"][l]
            for l in range(num_locations)
        ]

        # =========
        # 目的関数定義
        # =========
        # 充足率の平均の最大化
        average_fill_rate_cost = -((sum_poly(w) / len(w)) ** 2)

        # 充足率の分散の最小化
        variance_fill_rate_cost = (
            sum_poly(len(w), lambda i: w[i] ** 2) / len(w)
            - (sum_poly(w) / num_locations) ** 2
        )

        # 従業員の希望度最大化
        location_cost = -sum_poly(
            num_workers,
            lambda i: sum_poly(
                num_locations,
                lambda l: df_worker_loc.loc[i][idx2loc[l]] * location_variables[i][l],
            ),
        )

        # ==========
        # 制約条件定義
        # ==========
        # 従業員iは同時に1店舗のみ勤務できる
        location_constarints = sum(
            [equal_to(sum_poly(location_variables[i]), 1) for i in range(num_workers)]
        )

        # 各店舗の要求人数に等しい管理職を配置する
        req_manager_constraints = sum(
            [
                equal_to(
                    sum_poly(num_workers, lambda i: role_variables[i][0][l]),
                    df_req["manager"][l],
                )
                for l in range(num_locations)
            ]
        )
        req_submanager_constraints = sum(
            [
                equal_to(
                    sum_poly(num_workers, lambda i: role_variables[i][1][l]),
                    df_req["submanager"][l],
                )
                for l in range(num_locations)
            ]
        )

        # 各店舗の要求人数以上の従業員を配置する
        req_employee_constraints = sum(
            [
                greater_equal(
                    sum_poly(num_workers, lambda i: location_variables[i][l]),
                    df_req["employee"][l],
                )
                for l in range(num_locations)
            ]
        )

        # ============
        # 最適化モデル作成
        # ============
        # それぞれの目的関数の係数
        loc_priority = 1
        ave_fill_priority = 1
        var_fill_priority = 10

        # 目的関数
        cost_func = (
            loc_priority * location_cost
            + ave_fill_priority * average_fill_rate_cost
            + var_fill_priority * variance_fill_rate_cost
        )

        # 制約条件を表すペナルティ関数の重み
        constraint_weight = 20

        # 制約条件
        constraints = constraint_weight * (
            location_constarints
            + req_manager_constraints
            + req_submanager_constraints
            + req_employee_constraints
        )

        # 最適化モデル
        model = cost_func + constraints
        return model, location_variables, role_variables

    def decode(
        self,
        result,
        location_variables,
        role_variables,
        df_worker_loc,
        df_worker_skill,
        df_req,
        roles,
        locations,
        *args,
        **kwargs,
    ):
        # 制約条件チェック
        if len(result) == 0:
            raise RuntimeError("The given constraints are not satisfied")
        values = result[0].values
        energy = result[0].energy

        # 割当店舗に関する変数の解
        location_solutions = decode_solution(location_variables, values)

        # 割当店舗と役職に関する変数の解
        role_solutions = decode_solution(role_variables, values)

        (role_index_list, loc_index_list) = np.where(np.array(role_solutions) == 1)[1:]
        dict_df = defaultdict(list)

        for i, (j, l) in enumerate(zip(role_index_list, loc_index_list)):
            ## 配属勤務地
            worker_id = df_worker_loc.loc[i]["worker_id"]
            role = roles[j]
            loc = locations[l]
            dict_df["worker_id"].append(worker_id)
            dict_df["role"].append(role)
            dict_df["location"].append(loc)

        df_result = pd.DataFrame.from_dict(dict_df, orient="index").T
        return df_result

In [ ]:
# インスタンス化
problem = DemoStep2()
display(HBox([problem.run_btn]), HBox([problem.problem_out]))
problem.show_problem()

display(HBox([problem.problem_result_out]))

<br>
<br>

---

## **Step 3**

In Step 3, in addition to the "*each employee's work location preference*", "*the number of employees required for each store*" and "*required number of employees per position for each store*" considered in **Step 2**, employee assignment is performed to meet the culinary skill requirements for each store based on each employee's role and skill level for each culinary skill.

Let us consider a sushi restaurant chain as an example. Each employee may have culinary skills such as "filleting, nigiri, soup cooking, and a la carte preparation," and the level of each skill is also quantified. On the other hand, each restaurant also has a required skill level for each skill type, and employees are assigned to meet that level.

In addition, employees are assigned to either a "floor" or a "kitchen" role. The number of people who need to be assigned as floor staff is specified, and employees with zero-cooking skills are automatically assigned as floor staff members.

Clicking the "Run" button will compute and display the results of the assignment for each employee and store.

In [ ]:
class DemoStep3(BaseDemo, AmplifyProblem):
    name = "step3"

    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        problem_out = Output()
        problem_result_out = Output()
        problem_out.add_class(self.name)
        problem_result_out.add_class(self.name)
        self.problem_out = problem_out
        self.problem_result_out = problem_result_out

    def _show_problem(self, *args, **kwargs):
        # 各店舗の要求スキル情報の読み込み
        ## store_require.csv
        dict_req = dict(
            location=["tenjin", "hakata"],
            manager=[1, 1],
            submanager=[0, 1],
            filleting=[1, 1],  # 捌き
            nigiri=[1, 2],  # 握り
            soup=[2, 2],  # 汁物
            a_la_carte=[2, 2],  # 一品
            floor=[1, 1],  # ホール
        )
        df_req = pd.DataFrame.from_dict(dict_req, orient="index").T
        message = (
            "The required number of managers and submanagers for each store, ",
            "the required level of cooking skills, and the number of floor staff",
            "are shown below.",
        )
        print("\n".join(message))
        display(df_req)

        # 各従業員の勤務希望情報の読み込み
        dict_worker_loc = dict(
            worker_id=[0, 1, 2, 50, 43], tenjin=[2, 1, 1, 1, 1], hakata=[1, 2, 1, 1, 1]
        )
        df_worker_loc = pd.DataFrame.from_dict(dict_worker_loc, orient="index").T

        message = ("Each employee's work location preference is shown below.",)
        print("\n".join(message))
        display(df_worker_loc)

        # 各従業員のスキル情報の読み込み
        dict_worker_skill = dict(
            worker_id=[0, 1, 2, 50, 43],
            manager=[1, 1, 0, 0, 0],
            submanager=[1, 1, 0, 1, 1],
            employee=[1, 1, 1, 1, 1],
            filleting=[2, 2, 0, 1, 1],  # 捌き
            nigiri=[2, 2, 0, 2, 2],  # 握り
            soup=[2, 2, 0, 0, 0],  # 汁物
            a_la_carte=[2, 2, 0, 1, 1],  # 一品
        )
        df_worker_skill = pd.DataFrame.from_dict(dict_worker_skill, orient="index").T

        message = (
            "For manager and submanager, if the value is 1, it means that the position",
            "is available. On the other hand, filleting, nigiri, soup, and a la carte ",
            "represent the skill level of each employee. 0 means that the employee is not ",
            "allowed to work in the kitchen (i.e., only as floor staff).",
        )
        print("\n".join(message))
        display(df_worker_skill)

        self.df_req = df_req
        self.df_worker_loc = df_worker_loc
        self.df_worker_skill = df_worker_skill

    def _show_result(self, *args, **kwargs):
        print(
            "The output of the results, where each employee will work from the results."
        )
        df_result_employee, df_result_store = self.solve(self, *args, **kwargs)
        display(df_result_employee)

        message = "Below shows the fillrates to see how much of the required skills are met by each store."
        print("".join(message))
        display(df_result_store)

    def solve(self, *args, **kwargs):
        """問題を解く関数"""
        model, location_variables, role_variables, assign_variables = self.construct(
            self.df_worker_loc, self.df_req, self.df_worker_skill
        )
        result = self._solve(model)
        df_result_employee, df_result_store = self.decode(
            result,
            location_variables,
            role_variables,
            assign_variables,
            self.df_worker_loc,
            self.df_worker_skill,
            self.df_req,
            self.roles,
            self.locations,
            self.assigns,
            self.skills,
        )
        return df_result_employee, df_result_store

    def construct(self, df_worker_loc, df_req, df_worker_skill, *args, **kwargs):
        # =======
        # 対応関係
        # =======
        # 従業員id
        workers = df_worker_loc["worker_id"].values
        # 店舗名
        locations = df_req["location"].values
        # 役職名
        roles = ["manager", "submanager", "employee"]
        # 役割名
        assigns = ["floor", "kitchen"]
        # 調理スキル名
        skills = ["filleting", "nigiri", "soup", "a_la_carte"]

        self.roles = roles
        self.locations = locations
        self.assigns = assigns
        self.skills = skills

        # dictの作成
        idx2loc = dict((i, v) for i, v in enumerate(locations))
        loc2idx = dict((v, i) for i, v in enumerate(locations))
        idx2role = dict((i, v) for i, v in enumerate(roles))
        role2idx = dict((v, i) for i, v in enumerate(roles))
        idx2skill = dict((i, v) for i, v in enumerate(skills))
        skill2idx = dict((v, i) for i, v in enumerate(skills))

        # 各データ長を取得
        num_workers = len(workers)
        num_locations = len(locations)
        num_roles = len(roles)
        num_assigns = len(assigns)
        num_skills = len(skills)

        # =======
        # 変数定義
        # =======
        # 従業員iが役職jで店舗lに勤務することを表す変数
        role_variables = gen_symbols(BinaryPoly, num_workers, num_roles, num_locations)

        # 従業員iが役職hで店舗lに勤務することを表す変数
        assign_variables = gen_symbols(
            BinaryPoly,
            num_workers * num_roles * num_locations,
            shape=(num_workers, num_assigns, num_locations),
        )

        # =======
        # 変数固定
        # =======
        for i, l in product(range(num_workers), locations):
            worker_req = df_worker_loc.iloc[i][l]
            if worker_req == 0:
                # 全ての役職で店舗割当が不可
                for j in range(num_roles):
                    role_variables[i][j][loc2idx[l]] = BinaryPoly(0)
                # 全ての役割で店舗割当が不可
                for h in range(num_assigns):
                    assign_variables[i][h][loc2idx[l]] = BinaryPoly(0)

        for i, j in product(range(num_workers), roles):
            worker_skill = df_worker_skill.iloc[i][j]
            if worker_skill == 0:
                # 全ての店舗で役職が不可
                for l in range(num_locations):
                    role_variables[i][role2idx[j]][l] = BinaryPoly(0)

        for i in range(num_workers):
            if all(df_worker_skill.iloc[i][k] == 0 for k in skills):
                # 全ての店舗で役割(キッチン担当)が不可
                for l in range(num_locations):
                    assign_variables[i][1][l] = BinaryPoly(0)

        location_variables = [
            [
                sum_poly(num_assigns, lambda h: assign_variables[i][h][l])
                for l in range(num_locations)
            ]
            for i in range(num_workers)
        ]

        ## 充足率の計算
        w = [
            (
                sum_poly(
                    num_workers,
                    lambda i: df_worker_skill.iloc[i][idx2skill[k]]
                    * assign_variables[i][1][l],
                )
            )
            / df_req[idx2skill[k]][l]
            for k in range(num_skills)
            for l in range(num_locations)
        ]

        # =========
        # 目的関数定義
        # =========
        # 充足率の平均の最大化
        average_fill_rate_cost = -((sum_poly(w) / len(w)) ** 2)

        # 充足率の分散の最小化
        variance_fill_rate_cost = (
            sum_poly(len(w), lambda i: w[i] ** 2) / len(w) - (sum_poly(w) / len(w)) ** 2
        )

        # 従業員の希望度最大化
        location_cost = -sum_poly(
            num_workers,
            lambda i: sum_poly(
                num_locations,
                lambda l: df_worker_loc.loc[i][idx2loc[l]] * location_variables[i][l],
            ),
        )

        # ==========
        # 制約条件定義
        # ==========
        # 従業員iは同時に1店舗のみ勤務できる
        location_constarints = sum(
            [equal_to(sum_poly(location_variables[i]), 1) for i in range(num_workers)]
        )

        # 各店舗の要求人数に等しい管理職を配置する
        req_manager_constraints = sum(
            [
                equal_to(
                    sum_poly(num_workers, lambda i: role_variables[i][0][l]),
                    df_req["manager"][l],
                )
                for l in range(num_locations)
            ]
        )
        req_submanager_constraints = sum(
            [
                equal_to(
                    sum_poly(num_workers, lambda i: role_variables[i][1][l]),
                    df_req["submanager"][l],
                )
                for l in range(num_locations)
            ]
        )

        # 各店舗の要求人数以上のホール担当を配置する
        req_hall_constraints = sum(
            [
                equal_to(
                    sum_poly(num_workers, lambda i: assign_variables[i][0][l]),
                    df_req["floor"][l],
                )
                for l in range(num_locations)
            ]
        )

        # 変数Mと変数Pの関係
        role_assign_constraints = sum(
            [
                equal_to(
                    sum_poly(num_roles, lambda j: role_variables[i][j][l])
                    - sum_poly(num_assigns, lambda h: assign_variables[i][h][l]),
                    0,
                )
                for i in range(num_workers)
                for l in range(num_locations)
            ]
        )

        # ============
        # 最適化モデル作成
        # ============
        # それぞれの目的関数の係数
        loc_priority = 1
        ave_fill_priority = 1
        var_fill_priority = 10

        # 目的関数
        cost_func = (
            loc_priority * location_cost
            + ave_fill_priority * average_fill_rate_cost
            + var_fill_priority * variance_fill_rate_cost
        )

        # 制約条件を表すペナルティ関数の重み
        constraint_weight = 10

        # 制約条件
        constraints = constraint_weight * (
            location_constarints
            + req_manager_constraints
            + req_submanager_constraints
            + req_hall_constraints
            + role_assign_constraints
        )

        # 最適化モデル
        model = cost_func + constraints
        return model, location_variables, role_variables, assign_variables

    def decode(
        self,
        result,
        location_variables,
        role_variables,
        assign_variables,
        df_worker_loc,
        df_worker_skill,
        df_req,
        roles,
        locations,
        assigns,
        skills,
        *args,
        **kwargs,
    ):
        # 制約条件チェック
        if len(result) == 0:
            raise RuntimeError("The given constraints are not satisfied")
        values = result[0].values
        energy = result[0].energy

        # 割当店舗に関する変数の解
        location_solutions = decode_solution(location_variables, values)

        # 割当店舗と役職に関する変数の解
        role_solutions = decode_solution(role_variables, values)

        # 割当店舗と役職に関する変数の解
        assign_solutions = decode_solution(assign_variables, values)
        (role_index_list, loc_index_list) = np.where(np.array(role_solutions) == 1)[1:]
        dict_df = defaultdict(list)

        for i, (j, l) in enumerate(zip(role_index_list, loc_index_list)):
            ## 配属勤務地
            worker_id = df_worker_loc.loc[i]["worker_id"]
            role = roles[j]
            loc = locations[l]
            dict_df["worker_id"].append(worker_id)
            dict_df["role"].append(role)
            dict_df["location"].append(loc)

        df_result_employee = pd.DataFrame.from_dict(dict_df, orient="index").T
        (assign_list, loc_index_list) = np.where(np.array(assign_solutions) == 1)[1:]

        dict_result_loc = defaultdict(lambda: defaultdict(int))

        for i, (j, l) in enumerate(zip(assign_list, loc_index_list)):
            # kitchenならば
            assign = assigns[j]
            worker_id = df_worker_loc.loc[i]["worker_id"]
            loc = locations[l]

            if assign == "kitchen":
                # すべての調理スキルの足し算を行う。
                for skill in skills:
                    dict_result_loc[loc][skill] += df_worker_skill.loc[i][skill]

            else:
                dict_result_loc[loc]["floor"] += 1

        df_result_loc = pd.DataFrame.from_dict(dict_result_loc, orient="index")

        dict_result = defaultdict(defaultdict)

        for i in range(len(df_result_loc)):
            loc = df_result_loc.iloc[i].name
            for skill in df_result_loc.keys():
                require_num_skill = df_req[df_req["location"] == loc][skill].item()
                satisfy_num_skill = df_result_loc.iloc[i][skill].item()
                dict_result[loc][
                    f"{skill}_fillrate"
                ] = f"{satisfy_num_skill/require_num_skill}"

        df_result_store = pd.DataFrame.from_dict(dict_result, orient="index")
        return df_result_employee, df_result_store

In [ ]:
# インスタンス化
problem = DemoStep3()
display(HBox([problem.run_btn]), HBox([problem.problem_out]))
problem.show_problem()

display(HBox([problem.problem_result_out]))